# Code to gather data from $z > 5.5$ QSOs.

We want to obtain data from several surveys and catalogs in order to  
manipulate them and try to obtain meaningful correlations.

First, we import the packages to be used

To get the first line working, you need  
to run the following lines:

```bash
 conda install nodejs
 pip install ipympl
 pip install --upgrade jupyterlab
 jupyter labextension install @jupyter-widgets/jupyterlab-manager
 jupyter labextension install jupyter-matplotlib
 jupyter nbextension enable --py widgetsnbextension
 ```

In [1]:
%matplotlib inline
# Static plots
#%matplotlib ipympl
# Interactive plots
import numpy as np
# import matplotlib.cm as cm
# import matplotlib.pyplot as plt
# import matplotlib.colors as mcolors
# import matplotlib.patheffects as mpe
# import matplotlib.patheffects as path_effects
# from matplotlib.ticker import FormatStrFormatter
# from matplotlib.patches import Polygon
from astropy.io import fits
from astropy.table import Table
from astropy.table import Column
from astropy.table import MaskedColumn
from astropy.table import hstack
from astropy.table import vstack
from astropy.wcs import WCS
from astropy import units as u
# from astropy.visualization import hist
from astropy.coordinates import SkyCoord
from astroquery.simbad import Simbad
from astroquery.ned import Ned
import getpass

One function, to derive luminosity distances

In [2]:
def luminosity_distance(z_i, H0=70., WM=0.3, WV=0.7):
    z_i   = np.array([z_i], dtype='float64').flatten()
    c     = 299792.458 # velocity of light in km/sec
    h     = H0 / 100.
    WR    = 4.165E-5 / (h * h)   # includes 3 massless neutrino species, T0 = 2.72528
    WK    = 1 - WM - WR - WV
    azs   = 1.0 / (1 + z_i)
    DTT   = 0.0
    DCMR  = 0.0
    # do integral over a=1/(1+z) from az to 1 in n steps, midpoint rule
    n     = 1000  # number of points in integrals
    DL_Mpcs = np.zeros_like(z_i)
    for count, az in enumerate(azs):
        a     = az + (1 - az) * (np.arange(0, n) + 0.5) / n
        adot  = np.sqrt(WK + (WM / a) + (WR / (a * a)) + (WV * a * a))
        for i in range(n):
            # a    = az + (1 - az) * (i + 0.5) / n
            # adot = math.sqrt(WK + (WM / a) + (WR / (a * a)) + (WV * a * a))
            DTT  = DTT + 1. / adot[i]
            DCMR = DCMR + 1. / (a[i] * adot[i])
        DTT   = (1. - az) * DTT / n
        DCMR  = (1. - az) * DCMR / n
        # tangential comoving distance
        ratio = 1.00
        x     = np.sqrt(abs(WK)) * DCMR
        if x > 0.1:
            if WK > 0:
                ratio =  0.5 * (np.exp(x) - np.exp(-x)) / x 
            else:
                ratio = np.sin(x) / x
        else:
            y = x * x
        if WK < 0: y = -y
        ratio  = 1. + y / 6. + y * y / 120.
        DCMT   = ratio * DCMR
        DA     = az * DCMT
        DL     = DA / (az * az)
        DL_Mpc = (c / H0) * DL
        DL_Mpcs[count] = DL_Mpc
    return DL_Mpcs

Define the spectral index $\alpha$ from different sources  
to be used in the luminosity calculations (K-correction)

In [3]:
alpha_first = 0.5  # From FIRST data (Bornancini+2010)
alpha_RG    = 1.0  # For radio galaxies (Verkhodanov & Khabibullina, 2010)
alpha_alex  = 0.8  # Star-forming galaxies (Alexander+2003)
alpha_smol  = 0.7  # Mean value from VLA-COSMOS 3GHz sample (Smolčić et al. 2017)
alpha_butl  = 0.75  # From Butler et al., 2018

Choose one of the spectral indexes

In [4]:
alpha_used  = alpha_butl

---

### Reading data

Next step, reading our data.  
Most of the data files have been created using Topcat

In [5]:
machine  = getpass.getuser()
# cat_path = '/home/' + machine + '/Documentos/Data/'
cat_path = ''  # relative path to the same directory

Read data from FIRST+MILLIQUAS catalogs cross-matched.  

**MILLIQUAS + FIRST (SDSS)**   

Redshift values have been retrieved from the column `pipeline_redshift` in SDSS DR12  
The procedure to obtain these values is explained in **Bolton+2012**  

We also select sources with have explicitely data in both SDSS Quasar Catalog and  
in the FIRST survey (`first_match_flag = 1`).  

In order to get only the best redshift values, we select sources with  
`pipeline_redshift_flag = 0`.

Thus, we can use 9161 objects from the catalog.

In [6]:
hdu_list = fits.open(cat_path + 'tables_matches_milli_sdss_apr.fits');

# wise_milli  = Table(hdu_list[1].data);
sdss_milli  = Table(hdu_list[1].data);
# match_table = Table(hdu_list[3].data);  # Three tables cross-matched
hdu_list.close();

L_14GHz_filter = np.array((sdss_milli['L_14GHz'] > 0.0) * (sdss_milli['first_offset'] < 1.0)) ;  # sdss + milliquasar

#np.sum(L_14GHz_filter)
#sdss_milli['name', 'redshift', 'redshift_err', 'first_offset', 'spec_index'][L_14GHz_filter * np.array(sdss_milli['redshift'] > 5.5)].show_in_browser(jsviewer=True)

We want, also, to add $z > 6$ QSOs from the list in  
Table 3 in the review of **Inayoshi, Visbal, and Haiman, 2020**.  
Six of them have $z > 7$

Not all of them have $1.4$ GHz measurements. Others have  
measurements in different frequencies which can be translated  
into the desired frequency using, for instance, the relation  
from **Butler et al., 2018**:

$$S_{a} = S_{b} \times (\frac{\nu_{b}}{\nu_{a}})^{\alpha}$$  

We load the data from these sources. Fluxes from different frequencies than $1.4$ GHz are translated to the needed value.

In [7]:
high_z_zs        = np.char.replace(np.loadtxt(cat_path + 'high_z_qso_props.csv', usecols=[3],  dtype='str', delimiter=';'), ',', '.').astype(np.float)
high_z_zs_e      = np.char.replace(np.loadtxt(cat_path + 'high_z_qso_props.csv', usecols=[4],  dtype='str', delimiter=';'), ',', '.').astype(np.float)
high_z_14GHz     = np.char.replace(np.loadtxt(cat_path + 'high_z_qso_props.csv', usecols=[6],  dtype='str', delimiter=';'), ',', '.').astype(np.float)
high_z_14GHz_e   = np.char.replace(np.loadtxt(cat_path + 'high_z_qso_props.csv', usecols=[7],  dtype='str', delimiter=';'), ',', '.').astype(np.float)
high_z_3GHz      = np.char.replace(np.loadtxt(cat_path + 'high_z_qso_props.csv', usecols=[8],  dtype='str', delimiter=';'), ',', '.').astype(np.float)
high_z_3GHz_e    = np.char.replace(np.loadtxt(cat_path + 'high_z_qso_props.csv', usecols=[9],  dtype='str', delimiter=';'), ',', '.').astype(np.float)
high_z_15GHz     = np.char.replace(np.loadtxt(cat_path + 'high_z_qso_props.csv', usecols=[10], dtype='str', delimiter=';'), ',', '.').astype(np.float)
high_z_15GHz_e   = np.char.replace(np.loadtxt(cat_path + 'high_z_qso_props.csv', usecols=[11], dtype='str', delimiter=';'), ',', '.').astype(np.float)
high_z_250GHz    = np.char.replace(np.loadtxt(cat_path + 'high_z_qso_props.csv', usecols=[12], dtype='str', delimiter=';'), ',', '.').astype(np.float)
high_z_250GHz_e  = np.char.replace(np.loadtxt(cat_path + 'high_z_qso_props.csv', usecols=[13], dtype='str', delimiter=';'), ',', '.').astype(np.float)
high_z_mass_1450 = np.char.replace(np.loadtxt(cat_path + 'high_z_qso_props.csv', usecols=[14], dtype='str', delimiter=';'), ',', '.').astype(np.float)
high_z_names     = np.loadtxt('high_z_qso_props.csv', usecols=[0], dtype='str', delimiter=';')
high_z_lum_d     = luminosity_distance(high_z_zs) * 3.086e22  # in m
high_z_up_lim    = np.array([val == '<' for val in np.loadtxt('high_z_qso_props.csv', usecols=[5], dtype='str', delimiter=';')])

Accumulate values into one array except 250GHz data.  
Millimetre luminosities will be used separately since we cannot be completely  
sure that they represent, fully, non-thermal emission (from AGN) and not dust.

In [8]:
high_z_14   = high_z_14GHz + high_z_3GHz + high_z_15GHz
high_z_14_e = high_z_14GHz_e + high_z_3GHz_e + high_z_15GHz_e

---

### Calculate luminosities

Calculate luminosities (in W/Hz) for different datasets  
using the expression

$$L_{1.4\mathrm{GHz}} = 4 \pi \mathrm{d}^{2}_{L} f_{1.4\mathrm{GHz}} (1 + z)^{\alpha - 1}$$

which comes from Alexander et al. 2003

We can also obtain that luminosity from the flux in $3$ GHz as

$$L_{1.4\mathrm{GHz}} = 4 \pi \mathrm{d}^{2}_{L} {(\frac{3}{1.4})}^{\alpha} f_{3\mathrm{GHz}} (1 + z)^{\alpha - 1}$$

This expression comes from Delhaize et al. 2017.

In [9]:
L_21cm      = 4 * np.pi * (sdss_milli['D_lum'][L_14GHz_filter])**2 * sdss_milli['flux_20_cm'][L_14GHz_filter] * 1e-3 * 1e-26 * (1 + sdss_milli['redshift'][L_14GHz_filter])**(alpha_used - 1)
L_21cm_e    = np.abs(L_21cm) / sdss_milli['snr_20_cm'][L_14GHz_filter]

In [10]:
high_z_lum_14    = 4 * np.pi * high_z_lum_d**2 * high_z_14GHz  * 1e-6 * 1e-26 * (1 + high_z_zs)**(alpha_used - 1)
high_z_lum_3     = 4 * np.pi * high_z_lum_d**2 * high_z_3GHz   * 1e-6 * 1e-26 * (1 + high_z_zs)**(alpha_used - 1) * (3/1.4)**alpha_used
high_z_lum_15    = 4 * np.pi * high_z_lum_d**2 * high_z_15GHz  * 1e-6 * 1e-26 * (1 + high_z_zs)**(alpha_used - 1) * (1.5/1.4)**alpha_used
high_z_lum_250   = 4 * np.pi * high_z_lum_d**2 * high_z_250GHz * 1e-6 * 1e-26 * (1 + high_z_zs)**(alpha_used - 1) * (250/1.4)**(alpha_used)

Mix all luminosities (different bands) to obtain single value (adding zeroes).  
Millimetre luminosities will be used separately since we cannot be completely  
sure that they represent, fully, non-thermal emission (from AGN) and not dust.

In [11]:
high_z_lum_14GHz   = high_z_lum_14 + high_z_lum_3 + high_z_lum_15

We can also determine error values for these luminosities

In [12]:
high_z_lum_14GHz_e  = np.zeros_like(high_z_lum_14GHz)
high_z_lum_250GHz_e = np.zeros_like(high_z_lum_250)
for counter, element in enumerate(high_z_lum_14GHz):
    if element == 0: continue
    high_z_lum_14GHz_e[counter] = np.abs(element) * high_z_14_e[counter] / high_z_14[counter]
for counter, element in enumerate(high_z_lum_250):
    if element == 0: continue
    high_z_lum_250GHz_e[counter] = np.abs(element) * high_z_250GHz_e[counter] / high_z_250GHz[counter]

Create a filter to plot, when needed, only the sources which  
have mm data but not radio observations.

In [13]:
filter_250GHz = np.array((high_z_lum_250 > 0) * (high_z_lum_14GHz == 0))

Now, we can use the points we are interested in. Our sample from **Inayoshi et al., 2020** and the  
sources from **SDSS+FIRST** with $z>5.5$.

Another option to display the data is, instead of showing redshift in the  
horizontal axis, have the mass of the observed objects.

**Inayoshi et al., 2020** use the rest-frame UV magnitude $\mathrm{M}_{1450}$  
to calculate the mass as:

$$M = 10^{[-(\mathrm{M}_{1450} + 3.459) / 2.5]} [\mathrm{M}_{\odot}]$$

which yields, on average, the published virial mass estimates for those available.

In [14]:
upper_sdss_L          = L_21cm[np.array(sdss_milli['redshift'][L_14GHz_filter] > 5.5)]
upper_sdss_L_e        = L_21cm_e[np.array(sdss_milli['redshift'][L_14GHz_filter] > 5.5)]
upper_sdss_u_lim      = np.zeros_like(upper_sdss_L, dtype=np.bool)
upper_sdss            = sdss_milli[np.array(sdss_milli['redshift'] > 5.5) * L_14GHz_filter]
upper_sdss_z          = upper_sdss['redshift']
upper_sdss_z_e        = upper_sdss['redshift_err']
upper_sdss_f_20cm     = upper_sdss['flux_20_cm']  # mJy
upper_sdss_f_20cm_e   = upper_sdss['flux_20_cm'] / upper_sdss['snr_20_cm']  # mJy
upper_sdss_f_250GHz   = np.zeros_like(upper_sdss_L)
upper_sdss_f_250GHz_e = np.zeros_like(upper_sdss_L)
upper_sdss_L_250GHz   = np.zeros_like(upper_sdss_L)
upper_sdss_L_250GHz_e = np.zeros_like(upper_sdss_L)
upper_sdss_mass_1450  = np.zeros_like(upper_sdss_L)

In [15]:
large_sample_L         = np.append(upper_sdss_L, high_z_lum_14GHz[np.array(high_z_lum_14GHz>0)])  # W/Hz
large_sample_L_e       = np.append(upper_sdss_L_e, high_z_lum_14GHz_e[np.array(high_z_lum_14GHz>0)])  # W/Hz
large_sample_u_lim     = np.append(upper_sdss_u_lim, high_z_up_lim[np.array(high_z_lum_14GHz>0)])
large_sample_L_250     = np.append(upper_sdss_L_250GHz, high_z_lum_250[np.array(high_z_lum_14GHz>0)])
large_sample_L_250_e   = np.append(upper_sdss_L_250GHz_e, high_z_lum_250GHz_e[np.array(high_z_lum_14GHz>0)])
large_sample_f20cm     = np.append(upper_sdss_f_20cm, high_z_14[np.array(high_z_lum_14GHz>0)] * 1e-3)  # mJy
large_sample_f20cm_e   = np.append(upper_sdss_f_20cm_e, high_z_14_e[np.array(high_z_lum_14GHz>0)] * 1e-3)  # mJy
large_sample_f250GHz   = np.append(upper_sdss_f_250GHz, high_z_250GHz[np.array(high_z_lum_14GHz>0)] * 1e-3)  # mJy
large_sample_f250GHz_e = np.append(upper_sdss_f_250GHz_e, high_z_250GHz_e[np.array(high_z_lum_14GHz>0)] * 1e-3)  # mJy
large_sample_z         = np.append(upper_sdss_z, high_z_zs[np.array(high_z_lum_14GHz>0)])
large_sample_z_e       = np.append(upper_sdss_z_e, high_z_zs_e[np.array(high_z_lum_14GHz>0)])
large_sample_mass_1450 = np.append(upper_sdss_mass_1450, high_z_mass_1450[np.array(high_z_lum_14GHz>0)])  # M_sun

At this point, we also want to obtain more properties from the selected  
sources (**Inayoshi et al., 2020** + **SDSS+FIRST**). We will use `astroquery` to  
obtain information from `simbad`.

First, we obtain the names of our sources to query them.

In [16]:
large_sample_names = np.append(upper_sdss['name'], high_z_names[np.array(high_z_lum_14GHz>0)])

Then, we can query the database to obtain the desired data.  In this point,  
we also add more columns to be queried.

In [17]:
customSimbad   = Simbad()
initial_fields = customSimbad.get_votable_fields()

if 'coordinates' in initial_fields:
    customSimbad.remove_votable_fields('coordinates')
    customSimbad.add_votable_fields('ra(d)', 'dec(d)')
if 'z_value' not in initial_fields:
    customSimbad.add_votable_fields('z_value')
for band in ['B','V','R','I','J','K']:
    if f'fluxdata({band})' not in initial_fields:
        customSimbad.add_votable_fields(f'flux({band})', f'flux_error({band})')


result_table = customSimbad.query_objects(large_sample_names)

From this point, we merge the data from the query to `simbad` with the  
values from this notebook (**Inayoshi et al., 2020** and **SDSS+FIRST**).  
In order to do this, we convert the data into `astropy` columns, and then  
into `astropy` tables. They will be ready to be exported.

In [18]:
column_z_own        = MaskedColumn(large_sample_z, name='Z_OWN', unit='', description='Redshift from Inayoshi+2020 or SDSS+FIRST', fill_value=np.nan)
column_z_own_err    = MaskedColumn(large_sample_z_e, name='Z_OWN_ERR', unit='', description='Redshift error from Inayoshi+2020 or SDSS+FIRST', fill_value=np.nan)
column_L_14GHz      = MaskedColumn(large_sample_L, name='L_20CM', unit='W/Hz', description='Luminosity in 1.4 GHz', fill_value=np.nan)
column_L_14GHz_err  = MaskedColumn(large_sample_L_e, name='L_20CM_ERR', unit='W/Hz', description='Luminosity error in 1.4 GHz', fill_value=np.nan)
column_L_14GHz_up   = MaskedColumn(large_sample_u_lim, name='L_20CM_UP_LIM', dtype='bool', description='True if L_20CM is upper limit')
column_L_250GHz     = MaskedColumn(large_sample_L_250, name='L_250GHZ', unit='W/Hz', description='Luminosity in 250 GHz', fill_value=np.nan)
column_L_250GHz_err = MaskedColumn(large_sample_L_250_e, name='L_250GHZ_ERR', unit='W/Hz', description='Luminosity error in 250 GHz', fill_value=np.nan)
column_f_20cm       = MaskedColumn(large_sample_f20cm, name='F_20CM', unit='mJy', description='Flux in 20 cm', fill_value=np.nan)
column_f_20cm_err   = MaskedColumn(large_sample_f20cm_e, name='F_20CM_ERR', unit='mJy', description='Flux error in 20 cm', fill_value=np.nan)
column_f_250GHz     = MaskedColumn(large_sample_f250GHz, name='F_250GHZ', unit='mJy', description='Flux in 250 GHz', fill_value=np.nan)
column_f_250GHz_err = MaskedColumn(large_sample_f250GHz_e, name='F_250GHZ_ERR', unit='mJy', description='Flux error in 250 GHz', fill_value=np.nan)
column_mass_1450    = MaskedColumn(large_sample_mass_1450, name='MASS_1450', unit='Msun', description='Mass from mag_1450 (UV)', fill_value=np.nan)

In [19]:
result_table.add_columns([column_z_own, column_z_own_err, column_L_14GHz, column_L_14GHz_err, column_L_14GHz_up, column_L_250GHz, column_L_250GHz_err, column_f_20cm, column_f_20cm_err, column_f_250GHz, column_f_250GHz_err, column_mass_1450])

In [20]:
str_id = result_table['MAIN_ID'].astype('str')
result_table.replace_column('MAIN_ID', str_id)

In [21]:
copy_table = result_table.filled(fill_value=np.nan)

We write the table into a file. It can be `.fits`, `.votable`, etc.

In [22]:
#copy_table.write('high_z_qsos.ecsv', format='ascii.ecsv', overwrite=True, serialize_method='data_mask')

Query the objects of the table in other catalogs and services.

In [23]:
#from astroquery.heasarc import Heasarc
#Heasarc.query_mission_cols(mission='radio')
#tabb = Heasarc.query(large_sample_names, mission='radio', timeout=90)

In [24]:
customNed        = Ned()
fields_to_remove = ['No.', 'Photometry Measurement', 'Uncertainty', 'Units', 'Frequency', 'Significance', 'Published frequency', 'Frequency Mode', 'Coordinates Targeted', 'Spatial Mode', 'Qualifiers', 'Comments']

In [25]:
empty_counter = 0
res_tab       = {}
for name in large_sample_names:
    try:
        res_tab[name] = customNed.get_table(name)
        res_tab[name].remove_columns(fields_to_remove)
    except:
        res_tab[name] = Table()
        empty_counter += 1

In [26]:
result_table_copy     = result_table.copy()
for index, source_name in enumerate(large_sample_names):
    band_names_str    = []
    column_names_str  = []
    measure_names     = res_tab[source_name].colnames[1:]
    init_table        = Table([[str(source_name)]], names=['MAIN_ID'])
    for row in res_tab[source_name]:
        band_name_str = str(row['Observed Passband'].decode('utf-8'))
        if str(band_name_str) not in band_names_str:
            band_names_str.append(str(band_name_str))
            for measurement in measure_names:
                new_column_name = band_name_str.replace(' ', '_') + '_' + str(measurement).replace(' ', '_')
                if new_column_name not in column_names_str:
                    column_names_str.append(new_column_name)
                    #print(column_names_str[-2::])
                    temp_column = MaskedColumn(row[measurement], name=new_column_name)
                    #print(temp_column)
                    init_table.add_column(temp_column)
    init_table.remove_column('MAIN_ID')
    if index == 0:
        init_table_large  = init_table.copy()
    if index != 0:
        init_table_large  = vstack([init_table_large, init_table])
    #print(index)
result_table_copy = hstack([result_table_copy, init_table_large])

In [27]:
#result_table_copy['MAIN_ID', '1.4_GHz_(VLA)_Flux_Density', '1.4GHz_Flux_Density'].show_in_browser(jsviewer=True)

In [28]:
result_table_copy['1.4_GHz_(VLA)_Flux_Density', '1.4GHz_Flux_Density'].info(['attributes', 'stats'])

<Table length=52>
           name             dtype          mean                 std            min    max   n_bad
-------------------------- ------- -------------------- -------------------- ------- ------ -----
1.4_GHz_(VLA)_Flux_Density float64 0.006062857142857142 0.006415414638165785 2.6e-05 0.0186    45
       1.4GHz_Flux_Density float64  0.01780142857142857  0.02275488169198703 0.00127 0.0715    45


In [29]:
#for name in large_sample_names:
#    print(name)

In [30]:
52 - empty_counter

41